In [1]:
%matplotlib widget
import obspy
from obspy.signal.trigger import classic_sta_lta, recursive_sta_lta, aic_simple
from obspy.signal.trigger import plot_trigger
from obspy.signal.trigger import coincidence_trigger
from obspy.signal.cross_correlation import correlation_detector
import obspy.signal.trigger as trigger
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plot_working
import event_processing
import loading_stream_function
import matplotlib.ticker as ticker
from datetime import datetime
from datetime import timedelta

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
chan = 'GHZ'
#statlist = [ 'A01' ,'A03' , 'A05' , 'A07' , 'A09' , 'A11' , 'A13' , 'A15' , 'A17' , 'A19' ]
statlist = [ 'A01' ,'A02' , 'A03' , 'A04' , 'A05' , 'A06' , 'A07' , 'A08' , 'A09', 'A10' , 'A11', 'A13', 'A14', 'A19']#, 'A16', 'A17', 'A18', 'A19' ]
#statlist = [ 'B01' ,'B02' , 'B03' , 'B04' , 'B06' , 'B07' , 'B08' , 'B09','B10' , 'B11', 'B12', 'B13' , 'B19' , 'B20']#,'B14','B15','B16','B17','B18','B19', 'B20' ]

day = 15
hr = 15
start_time = obspy.UTCDateTime('2020-1-' + str(day) + 'T00:00:00')+hr*3600
end_time = obspy.UTCDateTime('2020-1-' + str(day) + 'T00:00:00')+(hr+1)*3600
stream = obspy.read("/media/sbrent/Oman3/PASSCAL/Main_deployment/DAYS/" +statlist[0]+ "/*.." + chan +".2020.0" +str(day))


In [4]:
stream[0].stats

         network: 7F
         station: A01
        location: 
         channel: BHZ
       starttime: 2020-01-15T00:00:00.000000Z
         endtime: 2020-01-15T23:59:59.999000Z
   sampling_rate: 1000.0
           delta: 0.001
            npts: 86400000
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'D', 'number_of_records': 21081, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 4096, 'filesize': 86347776})

In [5]:
for s in statlist[1:]:
    
    stream += obspy.read("/media/sbrent/Oman3/PASSCAL/Main_deployment/DAYS/" + s + "/*.." + chan + ".2020.0" + str(day))


In [6]:
data = stream.slice(starttime=start_time,endtime=end_time) #look into trim()

In [7]:
stream_filt = data.filter('highpass', corners=4, freq=80, zerophase=False)
stream_2 = stream_filt.copy()
#events = coincidence_trigger("classicstalta", 4, 0.5, stream_2, 3, sta=0.2, lta=4)
#events = coincidence_trigger("classicstalta", 10, 0.5, stream_2, 3, sta=0.08, lta=1)
#events = coincidence_trigger("classicstalta", 5, 0.5, stream_2, 3, sta=0.08, lta=1)
#events = coincidence_trigger("classicstalta", 5, 0.5, stream_2, 3, sta=0.08, lta=0.5) #best as of 08/24/2022 at 12:41
#events = coincidence_trigger("classicstalta", 5, 0.4, stream_2, 3, sta=0.08, lta=0.5)  # also good 08/24/2022 at 12:49
#events = coincidence_trigger("classicstalta", 5, 0.6, stream_2, 3, sta=0.08, lta=0.5) # also good 08/24/2022 at 12:52
#events = coincidence_trigger("classicstalta", 5, 0.7, stream_2, 3, sta=0.08, lta=0.5) # also good 08/24/2022 at 12:57
#events = coincidence_trigger("classicstalta", 5, 0.8, stream_2, 3, sta=0.08, lta=0.5)  # also good 08/24/2022 at 13:00

#events = coincidence_trigger("classicstalta", 5, 0.8, stream_2, 3, sta=0.06, lta=0.5) # very good
#events = coincidence_trigger("classicstalta", 5, 0.8, stream_2, 3, sta=0.04, lta=0.3) # even better
events = coincidence_trigger("classicstalta", 5, 0.8, stream_2, 3, sta=0.02, lta=0.2)




#events = coincidence_trigger("classicstalta", 4, 0.5, stream_2, 3, sta=0.08, lta=0.5)
events = pd.DataFrame(events)

In [ ]:
events

In [8]:
event_id = 147
event_meta = events.iloc[event_id]

In [9]:
gw_test = event_processing.get_waveforms_window(data, event_meta.time)

In [10]:
#aic_t, aics = event_processing.aic_pick(gw_test)
aic_t1, aics1 = event_processing.aic_pick_t1(gw_test)

In [14]:
len(aics1[0])

901

In [ ]:
aic_t, aics = event_processing.aic_pick(gw_test)

In [ ]:
gw_test

In [ ]:
event_processing.plot_waveforms_with_aic(gw_test,aics,aic_t)

In [ ]:
event_id = 147
event_meta = events.iloc[event_id]
start_window_edge = -0.1
end_window_edge = 0.25
starttime = event_meta['time'] + start_window_edge
endtime = event_meta['time'] + end_window_edge

In [ ]:
event = data.slice(starttime=starttime, endtime=endtime).copy()

In [ ]:
event_times = [t.times() for t in event]
event_data = [d.data for d in event]

In [ ]:
aic_score = [trigger.aic_simple(tr.data) for tr in event]

In [ ]:
diffs = [np.diff(aic, n=1) for aic in aic_score]
maxes = [np.argmax(diff) for diff in diffs]
aic_t = [event[n].times('matplotlib')[i] for n, i in enumerate(maxes)]

In [ ]:
#print(diffs)
print(maxes)
print(aic_t)

In [ ]:
fig, ax = plt.subplots(14, figsize=(10, 10), sharex=True)

ax = ax.flatten()

for n in np.arange(0, len(event), 1):
    ax[n].plot(event_times[n], event_data[n], color='black')
    
    event.taper(type='hann', max_percentage=0.5)
    ax[n].plot(event_times[n], event_data[n], color='black')
    
    n_zeros = 100
    aic_score = trigger.aic_simple(event_data[n])
    ax[n].vlines(event_times[n][aic_score.argmin()], ymin=10000, ymax=-10000, color='red')
    
    ax2 = ax[n].twinx()
    ax2.plot(event_times[n], aic_score)
    
fig.tight_layout()

In [ ]:
fig, axes = plot_working.plot_wave(event, color='black')
for n, ax in enumerate(axes):
    #aic = aic_score[n]
    #print(aic)
    ax2 = ax.twinx()
    t = event[n].times('matplotlib')
    #aic = aic_score[n]
    ax2.plot(t, aic_score[n], color='red')
    ax.plot((aic_t[n], aic_t[n]), (-2000, 2000), color='dodgerblue')


In [ ]:
axes_id = dict(np.array([['7F.A01..BHZ' ,'7F.A02..BHZ' , '7F.A03..BHZ' , '7F.A04..BHZ' , '7F.A05..BHZ' , '7F.A06..BHZ' , '7F.A07..BHZ' , '7F.A08..BHZ' , '7F.A09..BHZ'  , '7F.A10..BHZ' , '7F.A11..BHZ', '7F.A13..BHZ', '7F.A14..BHZ', '7F.A19..BHZ'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13], ]).transpose())

fig, ax = plt.subplots(len(event), figsize=(len(event), 15), sharex=True, sharey=True)

for i in np.arange(0, len(event), 1):

    ax[i].plot(event_times[i], event_data[i], color='black')
    
    event.taper(type='hann', max_percentage=0.5)
    ax[i].plot(event_times[i], event_data[i], color='blue')
    
    aic_score = trigger.aic_simple(event_data[i])
    #diffs = np.diff(aic_score[i])
    ax[i].vlines(event_times[i][aic_score.argmin()], ymin=10000, ymax=-10000, color='red')
    
    ax[i].text(1, 10000, list(axes_id.keys())[i])

#event.taper(type='hann', max_percentage=0.5)
#ax.plot(event.times(), event.data, color='black')

#aic = trigger.aic_simple(event.data)
#ax.vlines(event.times()[aic.argmin()], ymin=0.5, ymax=-0.5, color='red')

In [ ]:
aic = trigger.aic_simple(event_data[12])
print(len(aic))
#print(aic)

In [ ]:
aic_score[1]
print(len(aic_score))

In [ ]:
diffs = np.diff(aic, n=1)
print(len(diffs))

In [ ]:
event_times[12][aic.argmin()]

In [ ]:
aic[2].argmin()

In [ ]:
len(aic_t_idx)

In [ ]:
aic_t, aics = event_processing.aic_pick_t1(gw_test)